In [16]:
import pandas as pd
import numpy as np
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import t
from datetime import datetime, timedelta

In [22]:
event_date = datetime(2018, 6, 28)
estimation_window_start = event_date - timedelta(days=365)
estimation_window_end = event_date - timedelta(days=30)
event_window_start = event_date - timedelta(days=10)
event_window_end = event_date + timedelta(days=10)
stock_list = ['ATUS', 'AFTM', 'ATUS','AAPL']  # List of stocks to analyze

# # Fetch historical data for the stocks
# stock_data = yf.download(stock_list, start=estimation_window_start, end=event_window_end)

# # Save the stock data to a CSV file
# stock_data.to_csv('stock_data.csv')

In [23]:
combined_data = pd.DataFrame()

# Fetch historical data and additional company information for each stock
for ticker in stock_list:
    stock = yf.Ticker(ticker)
    
    # Fetch historical stock data
    historical_data = stock.history(start=estimation_window_start, end=event_window_end)
    historical_data['Ticker'] = ticker

    # Fetch additional company information
    info = stock.info
    # revenue = info.get('revenue', 'N/A')
    market_cap = info.get('marketCap', 'N/A')

    # Add the additional information to the historical data
    # historical_data['Revenue'] = revenue
    historical_data['MarketCap'] = market_cap

    # Append this stock's data to the combined DataFrame
    combined_data = pd.concat([combined_data, historical_data])

# Save the combined data to a CSV file
combined_data.to_csv('combined_stock_data2.csv')

In [8]:
# Initialize an empty DataFrame to store combined data
combined_data = pd.DataFrame()

# Fetch historical data and additional company information for each stock
for ticker in stock_list:
    stock = yf.Ticker(ticker)
    
    # Fetch historical stock data including dividends and stock splits
    historical_data = stock.history(start=estimation_window_start, end=event_window_end, actions=True)
    historical_data['Ticker'] = ticker

    # Append this stock's data to the combined DataFrame
    combined_data = pd.concat([combined_data, historical_data])

# Save the combined data to a CSV file
combined_data.to_csv('combined_stock_data_with_dividends.csv')

print(combined_data['Dividends'].unique())
# combined_data = pd.DataFrame()

# # Fetch historical data and additional company information for each stock
# for ticker in stock_list:
#     stock = yf.Ticker(ticker)
    
#     # Fetch historical stock data
#     historical_data = stock.history(start=estimation_window_start, end=event_window_end)
#     historical_data['Ticker'] = ticker


#     # Append this stock's data to the combined DataFrame
#     combined_data = pd.concat([combined_data, historical_data])

# # Save the combined data to a CSV file
# combined_data.to_csv('combined_stock_data.csv')

[0.     0.1825 0.1925]


In [ ]:
    # # Fetch additional company information
    # info = stock.info
    # revenue = info.get('revenue', 'N/A')
    # market_cap = info.get('marketCap', 'N/A')
    # net_income = info.get('netIncome', 'N/A') 

    # # Add the additional information to the historical data
    # historical_data['Revenue'] = revenue
    # historical_data['MarketCap'] = market_cap
    # historical_data['NetIncome'] = net_income


In [6]:
stock = yf.Ticker('AAPL')
info = stock.info
print(info)  # Check the content of info


{'address1': 'One Apple Park Way', 'city': 'Cupertino', 'state': 'CA', 'zip': '95014', 'country': 'United States', 'phone': '408 996 1010', 'website': 'https://www.apple.com', 'industry': 'Consumer Electronics', 'industryKey': 'consumer-electronics', 'industryDisp': 'Consumer Electronics', 'sector': 'Technology', 'sectorKey': 'technology', 'sectorDisp': 'Technology', 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In add